In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from petitRADTRANS import physical_constants as cst
import os
os.environ['OMP_NUM_THREADS'] = '1' # to avoid using too many CPUs, important for MPI
from mpi4py import MPI 
comm = MPI.COMM_WORLD # important for MPI
rank = comm.Get_rank() # important for MPI
import matplotlib
matplotlib.use('Agg') # disable interactive plotting

from system import System
from METIS import METIS
from crosscorr import CrossCorr
from utils import *

project_name = 'test'
chemistry = 'free' # or equ or Sorg20X
species_names = ['H2','He','H2O','CH4']

observation_properties = {'exptime_per_frame': 200*u.s,
                         'METIS_wave_range_um': [3,3.3],  # 0.3um wide
                         'pRT_wave_range_um': [2.8,3.7],} # wider range for cross-correlation

system_properties = {'planet_radius': 1.0 * cst.r_jup_mean, # 0.23* cst.r_jup_mean
                    'R_star': 0.5 * u.R_sun, # stellar radius
                    'd_star': 38 * u.pc, # distance to K2-18
                    'period': 10.0*u.day, #32.9*u.day real
                    'inclination': 89.57*u.deg,
                    'sma': 0.15*u.au,
                    'e': 0.0,
                    'transit_duration': 2.67*u.h}

atmosphere_properties = {'pressure': np.logspace(-6, 2, 100),
                        'temperature': 300,
                        'log_g': 3}

chemsitry_properties = {'species_names': species_names,
                        'chemistry' : chemistry}

if chemistry=='free':
    chemsitry_properties['log_H2O'] = -2
    chemsitry_properties['log_CH4'] = -2

elif chemistry=='equ':
    chemsitry_properties['Fe/H'] = 0.0
    chemsitry_properties['C/O'] = 0.59

elif chemistry in ['Sorg1X','Sorg20X']: # Hycean model with 1X or 20X organic sulfur
    tab = PSG_input(chemistry).table
    atmosphere_properties['pressure'] = tab['Pressure'].to_numpy()
    atmosphere_properties['temperature'] = tab['Temperature'].to_numpy()
    for species_i in species_names:
        chemsitry_properties[species_i] =  tab[species_i].to_numpy()

parameters = {**observation_properties, 
              **system_properties, 
              **atmosphere_properties, 
              **chemsitry_properties}
parameters['project_path'] = os.path.join(os.getcwd(),project_name)

system_obj = System(parameters,plot=True)

# for the future, center at ~3.30 µm (≈ 3.15–3.45 µm) for DMS, CH4, H2O
# worth including for L-band: H₂O, CH₄, NH₃, C₂H₂, C₂H₄, DMS
wavelength_range = parameters['METIS_wave_range_um']  # 0.3um
wave_width = 0.05 # determined wavelength range per simulation: 0.04674999999999985
n_wave_orders = (wavelength_range[-1]-wavelength_range[0])/wave_width
central_waves = np.arange(wavelength_range[0]+wave_width/2,wavelength_range[-1],wave_width)
transit_flux_array = system_obj.get_transit_array(wavelength_range,plot=True)

central_waves = [central_waves[0]] # for testing

plot_exp = 0
plot_order= 0
wl_obs, fl_obs, err_obs = [],[],[]
for i,central_wave in enumerate(central_waves):
    metis = METIS(central_wave,system_obj)
    transit, dark, flat, sky = metis.observe_transit_calib(system_obj.transit_flux_array,f'order{i}',plot_exp=plot_exp)
    wl, fl, err = metis.rectify_calibrate_extract(transit, dark, flat, sky, plot_exp=plot_exp)
    wl_obs.append(wl)
    fl_obs.append(fl)
    err_obs.append(err)

# reshape to (num_exp, num_orders, num_wave) makes more sense
wl_obs, fl_obs, err_obs = [np.array(var) for var in [wl_obs, fl_obs, err_obs]]
num_orders, num_exp, num_wave = fl_obs.shape
fl_obs, err_obs = [var.reshape((num_exp, num_orders, num_wave)) for var in [fl_obs, err_obs]]

py.warnings - WARNING: /home/grasser/.conda/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

py.warnings - WARNING: /home/grasser/.conda/envs/py311/lib/python3.11/site-packages/scopesim/effects/metis_lms_trace_list.py:409: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  thematrix[i, j] = (subpoly["P3"][sel_ij] * angle**3 +

py.warnings - WARNING: /home/grasser/.conda/envs/py311/lib/python3.11/site-packages/scopesim/effects/metis_lms_trace_list.py:291: DeprecationWarning: The fov_grid method is deprecated and will be removed in a future release. The functionality should be moved somewhere else.
  self.meta["fov"] = self.f

In [3]:
CrossCorr(system_obj, wl_obs, fl_obs, err_obs, plot_order=plot_order, template='H2O')


Loading Radtrans opacities...
 Loading line opacities of species 'H2O' from file '/net/lem/data2/pRT3_formatted/input_data/opacities/lines/line_by_line/H2O/1H2-16O/1H2-16O__POKAZATEL.R1e+06_0.3-28.0mu.xsec.petitRADTRANS.h5'... Done.
 Successfully loaded all line opacities
 Loading CIA opacities for H2--H2 from file '/net/lem/data2/pRT3_formatted/input_data/opacities/continuum/collision_induced_absorptions/H2--H2/H2--H2-NatAbund/H2--H2-NatAbund__BoRi.R831_0.6-250mu.ciatable.petitRADTRANS.h5'... Done.
 Loading CIA opacities for H2--He from file '/net/lem/data2/pRT3_formatted/input_data/opacities/continuum/collision_induced_absorptions/H2--He/H2--He-NatAbund/H2--He-NatAbund__BoRi.DeltaWavenumber2_0.5-500mu.ciatable.petitRADTRANS.h5'... Done.
 Successfully loaded all CIA opacities
Successfully loaded all opacities
